In [33]:
import pandas as pd
import os,re

# restaurant

In [36]:
def check_review(i, filePath, new_filePath):
    try:
        global dropna_, drop_duplicates_author_name, drop_duplicates_review_text
        global less_than_ten_word, more_than_five_hundred_word, before_clean_count, after_clean_count
        
        df = pd.read_csv(f"{filePath}/{i}.csv")
        df = df.iloc[:,1:]
        # 1 計算原評論筆數,並加回總數
        before_clean_count += len(df)
        
        # 2 計算na筆數,並加回總數
        dropna_ += df[df.loc[:,"review_text"].isna()].shape[0]
        # 刪沒寫評論只評星星
        df = df.dropna(subset="review_text")
        
        # 3 計算作者重複筆數,並加回總數
        drop_duplicates_author_name += df[df.duplicated(subset="author_name")].shape[0]
        # 刪作者重複
        df = df.drop_duplicates(subset=["author_name"])
           
        # 4 計算評論重複筆數,並加回總數
        drop_duplicates_review_text += df[df.duplicated(subset="review_text")].shape[0]
        # 刪評論重複,防範洗評論
        df = df.drop_duplicates(subset=["review_text"])
             
        # 5 把所有老鼠字眼都換成蟑螂,因模型判斷不出有老鼠是壞事
        df["review_text"] = df["review_text"].str.replace("老鼠","蟑螂")
        # 6 把非中文或英文的篩選掉,並把有"由 Google 提供翻譯"篩選掉
        regex = r"\\|\n|\.|\’|[\u0800-\u4e00]|\(由 Google 提供翻譯\)|[\uAC00-\uD7A3]"
        df["review_text"] = df["review_text"].apply(lambda strResult: re.sub(regex,"", strResult))
        # 7 把原始評論刪除,只留翻譯過後的中文
        df["review_text"] = df["review_text"].str.split("\(原始評論\)").str.get(0)
        # 8 計算中文字數的長度,不計算標點符號
        series = df["review_text"].str.findall("[\u4e00-\u9fa5]")
        series = series.str.len()
        
        # 9 計算中文字數小於等於10的筆數,並加回總數
        less_than_ten_word += df[series<=10].shape[0]
        
         # 只取中文字數大於10的筆數
        df = df[series>10]
        
        # 10 計算總字數包含標點符號大於500的筆數,並加回總數,因模型只接受500以下的字數
        more_than_five_hundred_word += df[df.loc[:,"review_text"].str.len() > 500].shape[0]
        
        # 只取總字數包含標點符號小於等於500的評論
        df = df[df.loc[:,"review_text"].str.len() <= 500]
               
        # 11 寫入csv
        df.to_csv(f"{new_filePath}/{i}.csv",index=None)
        after_clean_count+=len(df)
    except:
        print(i)

In [37]:
df = pd.read_csv("place_info_final_1229.csv")
filePath = "Data/raw_data/review_raw_res_att/"
new_filePath = "Data/raw_data/review_clean_res_att/"
file_name_1 = df.loc[:,"file_name_1"]

dropna_ = 0
drop_duplicates_author_name = 0
drop_duplicates_review_text = 0
less_than_ten_word = 0
more_than_five_hundred_word = 0
before_clean_count = 0
after_clean_count  = 0

for i in file_name_1:
    check_review(i, filePath, new_filePath)

1239
25145
25146
25147
25148
25149


In [38]:
print(f"沒評論只評星星共:{dropna_}筆")
print(f"同餐廳同個消費者評論卻重複共:{drop_duplicates_author_name}筆")
print(f"同餐廳不同消費者評論卻相同共:{drop_duplicates_review_text}筆")
print(f"評論字數小於10的評論共:{less_than_ten_word}筆")
print(f"評論含標點符號總數大於500的評論共:{more_than_five_hundred_word}筆")
print(f"before_clean_count共:{before_clean_count}筆")
print(f"after_clean_count共:{after_clean_count}筆")

沒評論只評星星共:3168520筆
同餐廳同個消費者評論卻重複共:47159筆
同餐廳不同消費者評論卻相同共:262249筆
評論字數小於10的評論共:1657767筆
評論含標點符號總數大於500的評論共:19400筆
before_clean_count共:8707405筆
after_clean_count共:3552310筆


In [42]:
8707405-3168520-47159-262249-1657767-19400

3552310

# attraction

In [40]:
df = pd.read_csv("attraction_info_final_1229.csv")
filePath = "Data/raw_data/review_raw_res_att/"
new_filePath = "Data/raw_data/review_clean_res_att/"
file_name_1 = df.loc[:,"file_name_1"]

dropna_ = 0
drop_duplicates_author_name = 0
drop_duplicates_review_text = 0
less_than_ten_word = 0
more_than_five_hundred_word = 0
before_clean_count = 0
after_clean_count  = 0

for i in file_name_1:
    check_review(i, filePath, new_filePath)

In [41]:
print(f"沒評論只評星星共:{dropna_}筆")
print(f"同餐廳同個消費者評論卻重複共:{drop_duplicates_author_name}筆")
print(f"同餐廳不同消費者評論卻相同共:{drop_duplicates_review_text}筆")
print(f"評論字數小於10的評論共:{less_than_ten_word}筆")
print(f"評論字數大於500的評論共:{more_than_five_hundred_word}筆")
print(f"before_clean_count共:{before_clean_count}筆")
print(f"after_clean_count共:{after_clean_count}筆")

沒評論只評星星共:539219筆
同餐廳同個消費者評論卻重複共:39524筆
同餐廳不同消費者評論卻相同共:25271筆
評論字數小於10的評論共:154051筆
評論字數大於500的評論共:817筆
before_clean_count共:972151筆
after_clean_count共:213269筆


In [43]:
972151-539219-39524-25271-154051-817

213269